In [147]:
import pandas as pd
import pytz

In [148]:
tempo = pd.read_csv('INMET_SE_SP_A701_SAO PAULO - MIRANTE_01-01-2019_A_31-12-2019.CSV', sep=';', header=8, encoding='utf-8')

In [149]:
tempo = tempo[['Data','Hora UTC','PRECIPITACAO TOTAL, HORARIO (mm)','TEMPERATURA MAXIMA NA HORA ANT. (AUT) (°C)','TEMPERATURA MINIMA NA HORA ANT. (AUT) (°C)','UMIDADE RELATIVA DO AR, HORARIA (%)','VENTO, VELOCIDADE HORARIA (m/s)']]
tempo = tempo.loc[(tempo['Data'] >= '2019/06/01') & (tempo['Data'] <= '2019/10/10')]

In [150]:

tempo['Datetime'] = pd.to_datetime(tempo['Data']+' '+tempo['Hora UTC'])

In [151]:
tempo['Datetime'] = tempo['Datetime'].dt.tz_convert(pytz.timezone('America/Sao_Paulo'))
tempo['Datetime'] = tempo['Datetime'].dt.tz_localize(None)

In [152]:
tempo["Temperatura Media"] = (tempo["TEMPERATURA MAXIMA NA HORA ANT. (AUT) (°C)"].str.replace(",", ".").astype(float) + tempo["TEMPERATURA MINIMA NA HORA ANT. (AUT) (°C)"].str.replace(",", ".").astype(float))/2

In [153]:
tempo.drop(["Data", "Hora UTC", "TEMPERATURA MAXIMA NA HORA ANT. (AUT) (°C)", "TEMPERATURA MINIMA NA HORA ANT. (AUT) (°C)"], axis=1, inplace=True)

In [154]:
tempo.columns = ["Precipitacao", "Umidade", "Vento", "Datetime", "Temperatura"]

In [155]:
tempo["Vento"] = tempo["Vento"].str.replace(",", ".").astype(float)
tempo["Precipitacao"] = tempo["Precipitacao"].str.replace(",", ".").astype(float)

In [156]:
tempo.dtypes

Precipitacao           float64
Umidade                  int64
Vento                  float64
Datetime        datetime64[ns]
Temperatura            float64
dtype: object

In [157]:
# tempo.to_json("Dados_clima.json")

In [158]:
df = pd.read_csv("rota33642.csv")

In [159]:
df["data_partida"] = pd.to_datetime(df["data_partida"])
df["data_partida"] = df["data_partida"].dt.round("H")

In [160]:
df_merged = df.merge(tempo, left_on="data_partida", right_on="Datetime", how="left")

In [161]:
df_merged.loc[df_merged["Temperatura"].isna()]

,data_partida,data_chegada,linha,tempo_viagem,equipamento,dia_semana,partidaTimeStamp,qtdDiasAno,direcao,tipo_dia,turno_dia,chegadaTimeStamp,hora,hora_dia,Precipitacao,Umidade,Vento,Datetime,Temperatura


In [162]:
df_merged.to_json("rota33642_clima.json")